# Getting Started

First, import the `bdikit` library.

In [1]:
import bdikit as bdi
import pandas as pd

In this example, we are mapping data from Dou et al. (https://pubmed.ncbi.nlm.nih.gov/37567170/) to the GDC format.

In [2]:
dataset = pd.read_csv("./datasets/dou.csv")

columns = [
    "Country",
    "Histologic_type",
    "FIGO_stage",
    "BMI",
    "Age",
    "Race",
    "Ethnicity",
    "Gender",
    "Tumor_Focality",
    "Tumor_Size_cm",
]

dataset[columns].head(10)

,Country,Histologic_type,FIGO_stage,BMI,Age,Race,Ethnicity,Gender,Tumor_Focality,Tumor_Size_cm
0,United States,Endometrioid,IA,38.88,64.0,White,Not-Hispanic or Latino,Female,Unifocal,2.9
1,United States,Endometrioid,IA,39.76,58.0,White,Not-Hispanic or Latino,Female,Unifocal,3.5
2,United States,Endometrioid,IA,51.19,50.0,White,Not-Hispanic or Latino,Female,Unifocal,4.5
3,NaN,Carcinosarcoma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,United States,Endometrioid,IA,32.69,75.0,White,Not-Hispanic or Latino,Female,Unifocal,3.5
5,United States,Serous,IA,20.28,63.0,White,Not-Hispanic or Latino,Female,Unifocal,6.0
6,United States,Endometrioid,IA,55.67,50.0,White,Not-Hispanic or Latino,Female,Unifocal,4.5
7,Other_specify,Endometrioid,IA,25.68,60.0,White,Not-Hispanic or Latino,Female,Unifocal,5.0
8,United States,Serous,IIIA,21.57,83.0,White,Not-Hispanic or Latino,Female,Unifocal,4.0
9,United States,Endometrioid,IA,34.26,69.0,White,Not-Hispanic or Latino,Female,Unifocal,5.2


### Matching the table schema to GDC standard vocabulary

`bdi-kit` offers a suite of functions to help with data harmonization tasks.
For instance, it can help with automatic discovery of one-to-one mappings between the columns in the input (source) dataset and a target dataset schema. The target schema can be either another table or a standard data vocabulary such as the GDC (Genomic Data Commons).

To achieve this using `bdi-kit`, we can use the `match_schema()` function to match columns to the GDC vocabulary schema as follows.

In [3]:
column_mappings = bdi.match_schema(dataset[columns], target="gdc", method="magneto_ft_bp")
column_mappings

/opt/miniconda3/envs/bdf/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


Loaded SentenceTransformer Model on cpu


,source,target,similarity
0,FIGO_stage,figo_stage,1.000000
1,Race,race,1.000000
2,Ethnicity,ethnicity,1.000000
3,BMI,bmi,1.000000
4,Gender,gender,1.000000
5,Tumor_Focality,tumor_focality,1.000000
6,Age,age_at_index,0.988827
7,Country,country_of_birth,0.957011
8,Tumor_Size_cm,tumor_length_measurement,0.905819
9,Histologic_type,histologic_progression_type,0.727179


### Generating a harmonized table

After discovering a schema mapping, we can generate a new table (DataFrame) using the new column names from the GDC standard vocabulary.

To do so using `bdi-kit`, we can use the function `materialize_mapping()` as follows. Note that the column headers have been renamed to the target schema.

In [4]:
bdi.materialize_mapping(dataset, column_mappings)

,figo_stage,race,ethnicity,bmi,gender,tumor_focality,age_at_index,country_of_birth,tumor_length_measurement,histologic_progression_type
0,IA,White,Not-Hispanic or Latino,38.88,Female,Unifocal,64.0,United States,2.9,Endometrioid
1,IA,White,Not-Hispanic or Latino,39.76,Female,Unifocal,58.0,United States,3.5,Endometrioid
2,IA,White,Not-Hispanic or Latino,51.19,Female,Unifocal,50.0,United States,4.5,Endometrioid
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carcinosarcoma
4,IA,White,Not-Hispanic or Latino,32.69,Female,Unifocal,75.0,United States,3.5,Endometrioid
...,...,...,...,...,...,...,...,...,...,...
99,IA,NaN,NaN,29.40,Female,Unifocal,75.0,Ukraine,4.2,Endometrioid
100,II,NaN,NaN,35.42,Female,Unifocal,74.0,Ukraine,1.5,Endometrioid
101,II,Black or African American,Not-Hispanic or Latino,24.32,Female,Unifocal,85.0,United States,3.8,Serous
102,IA,NaN,NaN,34.06,Female,Unifocal,70.0,Ukraine,5.0,Serous


### Generating a harmonized table with value mappings

`bdi-kit` can also help with translation of the values from the source table to the target standard format.

To this end, `bdi-kit` provides the function `match_values()` that automatically creates value mappings for each string column.
The output of `match_values()` can be fed to `materialize_mapping()` which materialized the final target using both schema and value mappings.

In [5]:
value_mappings = bdi.match_values(dataset, target="gdc", column_mapping=column_mappings, method="tfidf")
bdi.materialize_mapping(dataset, value_mappings)

,figo_stage,race,ethnicity,gender,tumor_focality,country_of_birth,histologic_progression_type
0,Stage IA,white,not hispanic or latino,female,Unifocal,United States,NaN
1,Stage IA,white,not hispanic or latino,female,Unifocal,United States,NaN
2,Stage IA,white,not hispanic or latino,female,Unifocal,United States,NaN
3,Unknown,american indian or alaska native,NaN,NaN,NaN,NaN,NaN
4,Stage IA,white,not hispanic or latino,female,Unifocal,United States,NaN
...,...,...,...,...,...,...,...
99,Stage IA,american indian or alaska native,NaN,female,Unifocal,Ukraine,NaN
100,Stage III,american indian or alaska native,NaN,female,Unifocal,Ukraine,NaN
101,Stage III,black or african american,not hispanic or latino,female,Unifocal,United States,NaN
102,Stage IA,american indian or alaska native,NaN,female,Unifocal,Ukraine,NaN


### Verifying the schema mappings

Sometimes the mappings generated automatically may be incorrect or you may to want verify them individually.
To verify the suggested column mappings, you can use `bdi-kit` and [bdi-viz](https://github.com/VIDA-NYU/bdi-viz), which offers additional APIs to visualize the data and make any modifications when necessary. 

For this example, we will use the column `Histologic_type`. We can start by exploring the columns most similar to `Histologic_type`. 

For this, we can use the `rank_schema_matches()` function. Here, we notice that `primary_diagnosis` could be a potential target column.


In [6]:
hist_type_matches = bdi.rank_schema_matches(dataset, target="gdc", columns=["Histologic_type"])
hist_type_matches

Table features loaded for 1 columns
Table features loaded for 736 columns


,source,target,similarity
0,Histologic_type,sample_type,0.554611
1,Histologic_type,history_of_tumor_type,0.542955
2,Histologic_type,primary_diagnosis,0.526662
3,Histologic_type,morphologic_architectural_pattern,0.478859
4,Histologic_type,viral_hepatitis_serologies,0.469105
5,Histologic_type,analyte_type_id,0.462921
6,Histologic_type,histone_variant,0.461407
7,Histologic_type,cog_rhabdomyosarcoma_risk_group,0.425261
8,Histologic_type,tumor_descriptor,0.419358
9,Histologic_type,specimen_type,0.419077


### Viewing the column domains

To verify that `primary_diagnosis` is a good target column, we view and compare the domains of each column using the `preview_domain()` function. For the source table, it returns the list of unique values in the source column. For the GDC target, it returns the list of unique valid values that a column can have.

Here we see that the values seem to be related.

In [7]:
bdi.preview_domain(dataset, "Histologic_type")

,value_name
0,Endometrioid
1,Carcinosarcoma
2,Serous
3,Clear cell


In [8]:
bdi.preview_domain("gdc", "primary_diagnosis")

,value_name,value_description,column_description
0,Abdominal desmoid,An insidious poorly circumscribed neoplasm ari...,Text term used to describe the patient's histo...
1,Abdominal fibromatosis,An insidious poorly circumscribed neoplasm ari...,
2,Achromic nevus,A benign nevus characterized by the absence of...,
3,Acidophil adenocarcinoma,A malignant epithelial neoplasm of the anterio...,
4,Acidophil adenoma,An epithelial neoplasm of the anterior pituita...,
...,...,...,...
2620,Wolffian duct tumor,An epithelial neoplasm of the female reproduct...,
2621,Xanthofibroma,A benign neoplasm composed of fibroblastic spi...,
2622,Yolk sac tumor,A non-seminomatous malignant germ cell tumor c...,
2623,Unknown,"Not known, not observed, not recorded, or refu...",


Since `primary_diagnosis` looks like a correct match for `Histologic_type`, we can modify the `column_mappings` variable directly.

In [9]:
column_mappings.loc[column_mappings["source"] == "Histologic_type", "target"] = "primary_diagnosis"
column_mappings

,source,target,similarity
0,FIGO_stage,figo_stage,1.000000
1,Race,race,1.000000
2,Ethnicity,ethnicity,1.000000
3,BMI,bmi,1.000000
4,Gender,gender,1.000000
5,Tumor_Focality,tumor_focality,1.000000
6,Age,age_at_index,0.988827
7,Country,country_of_birth,0.957011
8,Tumor_Size_cm,tumor_length_measurement,0.905819
9,Histologic_type,primary_diagnosis,0.727179


### Finding correct value mappings

After finding the correct column, we need to find appropriate value mappings. 
Using `match_values()`, we can inspect what the possible value mappings for this would look like after the harmonization.

`bdi-kit` implements multiple methods for value mapping discovery, including:

 - `edit_distance` - Computes value similarities using Levenstein's edit distance measure.
 - `tfidf` - A method based on tf-idf importance weighting computed over charcter n-grams.
 - `embeddings` - Uses BERT word embeddings to compute "semantic similarity" between the values.

To specify a value mapping approach, we can pass the `method` parameter.

In [10]:
bdi.match_values(
    dataset, target="gdc", column_mapping=("Histologic_type", "primary_diagnosis"), method="edit_distance"
)

,source,target,similarity
0,Carcinosarcoma,"Carcinosarcoma, NOS",0.848485
1,Clear cell,Clear cell adenoma,0.714286
2,Endometrioid,"Endometrioid adenoma, NOS",0.648649
3,Serous,Neuronevus,0.625000


In [11]:
bdi.match_values(
    dataset, target="gdc", column_mapping=("Histologic_type", "primary_diagnosis"), method="tfidf"
)

,source,target,similarity
0,Carcinosarcoma,"Carcinosarcoma, NOS",0.969
1,Endometrioid,"Endometrioid adenoma, NOS",0.897
2,Clear cell,Clear cell adenoma,0.853
3,Serous,"Serous carcinoma, NOS",0.755


In [12]:
bdi.match_values(
    dataset, target="gdc", column_mapping=("Histologic_type", "primary_diagnosis"), method="embedding"
)

,source,target,similarity
0,Carcinosarcoma,Carcinofibroma,0.897
1,Clear cell,Clear cell carcinoma,0.773
2,Endometrioid,Endometrioid cystadenocarcinoma,0.755
3,Serous,Myoma,0.647


In [14]:
hist_type_vmap = pd.DataFrame(
    columns=["source", "target"],
    data=[
        ("Carcinosarcoma", "Carcinosarcoma, NOS"),
        ("Clear cell", "Clear cell adenocarcinoma, NOS"),
        ("Endometrioid", "Endometrioid carcinoma"),
        ("Serous", "Serous cystadenocarcinoma"),
    ],
)
hist_type_vmap

,source,target
0,Carcinosarcoma,"Carcinosarcoma, NOS"
1,Clear cell,"Clear cell adenocarcinoma, NOS"
2,Endometrioid,Endometrioid carcinoma
3,Serous,Serous cystadenocarcinoma


### Verifying multiple value mappings at once

Besides verifying value mappings individually, you can also do it for all column mappings at once.

In [15]:
mappings = bdi.match_values(
    dataset,
    target="gdc",
    column_mapping=column_mappings,
    method="tfidf",
)

for mapping in mappings:
    print(f"{mapping.attrs['source']} => {mapping.attrs['target']}")
    display(mapping)
    print("")

FIGO_stage => figo_stage


,source,target,similarity
0,IIIC2,Stage IIIC2,0.889
1,IIIC1,Stage IIIC1,0.889
2,IVB,Stage IVB,0.854
3,IIIB,Stage IIIB,0.849
4,IIIA,Stage IIIA,0.822
5,II,Stage III,0.687
6,IB,Stage IB,0.649
7,IA,Stage IA,0.586
8,NaN,Unknown,0.350



Race => race


,source,target,similarity
0,White,white,1.000
1,White,white,1.000
2,Asian,asian,1.000
3,Not Reported,not reported,1.000
4,Black or African American,black or african american,1.000
5,NaN,american indian or alaska native,0.345



Ethnicity => ethnicity


...

,source,target,similarity
0,Unifocal,Unifocal,1.0
1,Multifocal,Multifocal,1.0
2,NaN,NaN,NaN



Country => country_of_birth


,source,target,similarity
0,United States,United States,1.0
1,Ukraine,Ukraine,1.0
2,Poland,Poland,1.0
3,Other_specify,NaN,NaN
4,NaN,NaN,NaN



Histologic_type => primary_diagnosis


,source,target,similarity
0,Carcinosarcoma,"Carcinosarcoma, NOS",0.969
1,Endometrioid,"Endometrioid adenoma, NOS",0.897
2,Clear cell,Clear cell adenoma,0.853
3,Serous,"Serous carcinoma, NOS",0.755


### Fixing remaining value mappings

We need fix a few value mappings:
- Race
- Tumor_Site

For race, we need to fix: `NaN` -> `american indian or alaska native`. Notice that there are two nearly identical values: "White" and " White" (the latter includes a leading blank space).

In [16]:
race_vmap = bdi.match_values(
    dataset,
    target="gdc",
    column_mapping=("Race", "race"),
    method="tfidf",
)
race_vmap

,source,target,similarity
0,White,white,1.000
1,White,white,1.000
2,Asian,asian,1.000
3,Not Reported,not reported,1.000
4,Black or African American,black or african american,1.000
5,NaN,american indian or alaska native,0.345


In [17]:
race_vmap = race_vmap[race_vmap["similarity"] >= 1.0]
race_vmap

,source,target,similarity
0,White,white,1.0
1,White,white,1.0
2,Asian,asian,1.0
3,Not Reported,not reported,1.0
4,Black or African American,black or african american,1.0


For `Tumor_Site`, given that this dataset is about endometrial cancer, all values must be mapped to "Endometrium". So instead of fixing each mapping individually, we will write a custom function that returns "Endometrium" regardless of the input value. Later, we will show how to use this function to transform the dataset.

In [18]:
bdi.match_values(
    dataset, target="gdc", column_mapping=("Tumor_Site", "tissue_or_organ_of_origin"), method="tfidf"
)

,source,target,similarity
0,Anterior endometrium,Endometrium,0.852
1,Posterior endometrium,Endometrium,0.823
2,"Other, specify",Other specified parts of pancreas,0.543
3,NaN,Anal canal,0.301


In [19]:
# Custom mapping function that will be used to map the values of the 'Tumor_Site' column
def map_tumor_site(source_value):
    return "Endometrium"

### Combining custom user mappings with suggested mappings

Before generating a final harmonized dataset, we can combine the automatically generated value mappings with the fixed mappings provided by the user. To do so, we use `bdi.merge_mappings()` function, which take a list of mappings (e.g., generated automatically) and a list of "user-defined mapping overrides" that will be combined with the first list of mappings and will take precedence whenever they conflict.

In our example below, all mappings specified in the variable `user_mappings` will override the mappings in `value_mappings` generated by the `bdi.match_values()` function.

In [20]:
from math import ceil

user_mappings = [
    {
        # When no mapping is need, specifying the source and target is enough
        "source": "BMI",
        "target": "bmi",
    },
    {
        "source": "Tumor_Size_cm",
        "target": "tumor_largest_dimension_diameter",
    },
    {
        # mapper can be a custom Python function
        "source": "Tumor_Site",
        "target": "tissue_or_organ_of_origin",
        "mapper": map_tumor_site,
    },
    {
        # Lambda functions can also be used as mappers
        "source": "Age",
        "target": "days_to_birth",
        "mapper": lambda age: -age * 365.25,
    },
    {
        "source": "Age",
        "target": "age_at_diagnosis",
        "mapper": lambda age: float("nan") if pd.isnull(age) else ceil(age*365.25),
    },
    {
        # We can also use a data frame to specify value mappings using the `matches` attribute
        "source": "Histologic_type",
        "target": "primary_diagnosis",
        "matches": hist_type_vmap
    },
    # For dataframes that contain the 'source' and 'target' columns as attributes,
    # such as the ones returned by the match_values() function, we can directly
    # use them as mappings
    race_vmap,
]


harmonization_spec = bdi.merge_mappings(value_mappings, user_mappings)


Finally, we generate the harmonized dataset, with the user-defined value mappings.

In [21]:
harmonized_dataset = bdi.materialize_mapping(dataset, harmonization_spec)
harmonized_dataset

,bmi,tumor_largest_dimension_diameter,tissue_or_organ_of_origin,days_to_birth,age_at_diagnosis,primary_diagnosis,race,figo_stage,ethnicity,gender,tumor_focality,country_of_birth,histologic_progression_type
0,38.88,2.9,Endometrium,-23376.00,23376.0,Endometrioid carcinoma,white,Stage IA,not hispanic or latino,female,Unifocal,United States,NaN
1,39.76,3.5,Endometrium,-21184.50,21185.0,Endometrioid carcinoma,white,Stage IA,not hispanic or latino,female,Unifocal,United States,NaN
2,51.19,4.5,Endometrium,-18262.50,18263.0,Endometrioid carcinoma,white,Stage IA,not hispanic or latino,female,Unifocal,United States,NaN
3,NaN,NaN,Endometrium,NaN,NaN,"Carcinosarcoma, NOS",NaN,Unknown,NaN,NaN,NaN,NaN,NaN
4,32.69,3.5,Endometrium,-27393.75,27394.0,Endometrioid carcinoma,white,Stage IA,not hispanic or latino,female,Unifocal,United States,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,29.40,4.2,Endometrium,-27393.75,27394.0,Endometrioid carcinoma,NaN,Stage IA,NaN,female,Unifocal,Ukraine,NaN
100,35.42,1.5,Endometrium,-27028.50,27029.0,Endometrioid carcinoma,NaN,Stage III,NaN,female,Unifocal,Ukraine,NaN
101,24.32,3.8,Endometrium,-31046.25,31047.0,Serous cystadenocarcinoma,black or african american,Stage III,not hispanic or latino,female,Unifocal,United States,NaN
102,34.06,5.0,Endometrium,-25567.50,25568.0,Serous cystadenocarcinoma,NaN,Stage IA,NaN,female,Unifocal,Ukraine,NaN


For comparison, here is how our original data looked like:

In [22]:
original_columns = map(lambda m: m["source"], harmonization_spec)
dataset[original_columns]

,BMI,Tumor_Size_cm,Tumor_Site,Age,Age,Histologic_type,Race,FIGO_stage,Ethnicity,Gender,Tumor_Focality,Country,Histologic_type
0,38.88,2.9,Anterior endometrium,64.0,64.0,Endometrioid,White,IA,Not-Hispanic or Latino,Female,Unifocal,United States,Endometrioid
1,39.76,3.5,Posterior endometrium,58.0,58.0,Endometrioid,White,IA,Not-Hispanic or Latino,Female,Unifocal,United States,Endometrioid
2,51.19,4.5,"Other, specify",50.0,50.0,Endometrioid,White,IA,Not-Hispanic or Latino,Female,Unifocal,United States,Endometrioid
3,NaN,NaN,NaN,NaN,NaN,Carcinosarcoma,NaN,NaN,NaN,NaN,NaN,NaN,Carcinosarcoma
4,32.69,3.5,"Other, specify",75.0,75.0,Endometrioid,White,IA,Not-Hispanic or Latino,Female,Unifocal,United States,Endometrioid
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,29.40,4.2,"Other, specify",75.0,75.0,Endometrioid,NaN,IA,NaN,Female,Unifocal,Ukraine,Endometrioid
100,35.42,1.5,"Other, specify",74.0,74.0,Endometrioid,NaN,II,NaN,Female,Unifocal,Ukraine,Endometrioid
101,24.32,3.8,"Other, specify",85.0,85.0,Serous,Black or African American,II,Not-Hispanic or Latino,Female,Unifocal,United States,Serous
102,34.06,5.0,"Other, specify",70.0,70.0,Serous,NaN,IA,NaN,Female,Unifocal,Ukraine,Serous
